# Creating a self attention layer

1. Tokenize words
2. Create a dense embedding
3. Matrix multiplication to create weights for scale
4. Softmax to regularize
5. Matrix multiplication weights • value

In [1]:
from transformers import AutoTokenizer

In [2]:
model_ckpt = "bert-base-uncased"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [4]:
text = "time flies like an arrow"

# Tokenize the text to extract input IDs

Each token in sentence has been mapped to a unique ID. Excluded the [CLS] and [SEP] tokens in the false argument. 

In [5]:
inputs = tokenizer(text, return_tensors = "pt", add_special_tokens=False)

In [6]:
inputs.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])

# Create dense embeddings

In [7]:
from torch import nn

In [8]:
from transformers import AutoConfig

In [9]:
config = AutoConfig.from_pretrained(model_ckpt)

In [10]:
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)

In [11]:
token_emb

Embedding(30522, 768)

# Generate embeddings by feeding in the input IDs

In [12]:
input_embeds = token_emb(inputs.input_ids)

In [13]:
input_embeds

tensor([[[ 1.5097, -0.6167,  0.8365,  ..., -0.5363, -1.1691, -0.4326],
         [ 0.9660, -0.4311,  0.6420,  ...,  0.1471, -1.0959, -0.7024],
         [-1.3677, -0.6547,  1.0896,  ...,  0.9842, -0.0391, -0.2675],
         [-1.1296,  0.8480,  1.4288,  ...,  1.6103,  0.0340,  0.1342],
         [ 0.5584, -0.1909, -0.6854,  ...,  0.6165, -1.6752,  0.2127]]],
       grad_fn=<EmbeddingBackward0>)

This has given us a tensor of shape [batch_size, seq_len, hidden_dim]

In [14]:
input_embeds.size()

torch.Size([1, 5, 768])

# Create the Query, Key, and Value Vectors and calculate attention scores using the dot product as the similarity function

In [15]:
import torch

In [16]:
from math import sqrt

In [17]:
query = key = value = input_embeds

In [18]:
dim_k = key.size(-1) # last value of tensor
print(dim_k)

768


## torch.bmm performs a batch matrix-matrix product of matrices stored in input and mat2.

https://pytorch.org/docs/stable/generated/torch.bmm.html

In [19]:
scores = torch.bmm(query, key.transpose(1,2))/ sqrt(dim_k)

In [20]:
scores.size()

torch.Size([1, 5, 5])

This has created a 5 x 5 matrix of attention scores per sample in the batch.

# Apply softmax

In [21]:
import torch.nn.functional as F

In [22]:
weights = F.softmax(scores, dim = -1)

In [23]:
weights.sum(dim=-1)

tensor([[1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)

# multiply the attention weights by the values

In [24]:
# value is the same as the inputs_embeds
attn_outputs = torch.bmm(weights, value)

In [25]:
attn_outputs.shape

torch.Size([1, 5, 768])

# wrap in function

In [39]:
def scaled_dot_production_attention(query, key, value):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1,2))/ sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)

# Creating multi-head attention 

# First code u a single attention head

Initialize three independent linear layers that apply matrix multiplication to the embedding vectors to produce tensors of shape [batch_size, seq_len, head_dim], where **head_dim** is the number of dimensions we are projecting into

In [40]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)
        

    def forward(self, hidden_state):
        attn_outputs = scaled_dot_production_attention(self.q(hidden_state), self.k(hidden_state), self.v(hidden_state))
        return attn_outputs

# Concatenate the outputs of each single attention head to implement the full multi-head attention layer

In [41]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim//num_heads
        self.heads = nn.ModuleList([AttentionHead(embed_dim, head_dim) for _ in range(num_heads)])
        self.output_linear = nn.Linear(embed_dim, embed_dim)
        
    def forward(self, hidden_state):
        x = torch.cat([h(hidden_state) for h in self.heads], dim = -1)
        x = self.output_linear(x)
        return(x)

# Check if multi-head attention layer produces the expected shape of our inputs

In [42]:
multihead_attn = MultiHeadAttention(config)

In [45]:
attn_output=multihead_attn(input_embeds)

In [47]:
attn_output.size() # want this to match the same shape as our input

torch.Size([1, 5, 768])

# Visualize

In [48]:
from bertviz import head_view

In [49]:
from transformers import AutoModel

In [50]:
model = AutoModel.from_pretrained(model_ckpt, output_attentions = True)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [51]:
sentence_a = "time flies like an arrow"

In [52]:
sentence_b = "fruit flies like a banana"

In [53]:
viz_inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt')

In [54]:
attention = model(**viz_inputs).attentions

In [55]:
sentence_b_start = (viz_inputs.token_type_ids == 0).sum(dim=1)

In [56]:
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])

In [57]:
head_view(attention, tokens, sentence_b_start, heads = [8])

<IPython.core.display.Javascript object>